# Aula 2: Pipeline, Validação e Treinamento (com validação pandas) para Fake News

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import joblib as joblib
import mlflow as mlflow
import os as os

In [ ]:
# Passo 1: Carregue os dados
caminho = "../data/news_limpo.csv"
if not os.path.exists(caminho): 
    raise FileNotFoundError(f"Arquivo não encontrado: {caminho}")

df = pd.read_csv(caminho)

In [ ]:
# exibir contagem de valores nulos por coluna
df.isnull().sum()

# exibir linhas cuja coluna title é nula
df[df["title"].isnull()]

In [ ]:
# Passo 2: Validação simples com pandas
assert df["title"].notnull().all(), "Coluna 'title' possui valores nulos." 
assert df["label"].notnull().all(), "Coluna 'label' possui valores nulos."
assert df["title"].apply(lambda x: isinstance(x, str)).all(), "Coluna 'title' não é do tipo string."
assert df["label"].isin(["fake", "real"]).all(), "Coluna 'label' possui valores inesperados."
print("Validação dos dados concluída com sucesso!")

In [ ]:
# Passo 3: Separe treino e teste
X = df['title'].fillna('')
y = df['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
# Passo 4: Crie e treine o pipeline
pipeline = Pipeline([
    ("vect", TfidfVectorizer(stop_words='english', max_df=0.7)),
    ("clf", LogisticRegression())
])
pipeline.fit(X_train, y_train)

In [ ]:
# Passo 5: Avalie o modelo
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred, labels=["fake", "real"]))

Salvar o modelo e vetor

In [ ]:
joblib.dump(pipeline.named_steps["clf"], "../model.joblib")
joblib.dump(pipeline.named_steps["vect"], "../vectorizer.joblib")

Rastrear os experimentos com MLFLOW

In [ ]:
from mlflow.models import infer_signature

input_example = pd.DataFrame({"title": ["This is a real news!"]})

mlflow.sklearn.log_model(
    sk_model=pipeline, 
    name="modelo_fake_news", 
    input_example=input_example
)